In [1]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from data import BodyPart 
import tensorflow as tf
import cv2

In [2]:

# loading final csv file
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    
    return X, y, classes

In [3]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center

In [4]:
def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.

    It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size


In [5]:
def normalize_pose_landmarks(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks

In [6]:
def landmarks_to_embedding(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    # Flatten the normalized landmark coordinates into a vector
    embedding = keras.layers.Flatten()(landmarks)
    return embedding

In [7]:
def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)


X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')





In [8]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(256, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)

In [9]:
X, y, class_names = load_csv('train_data.csv')
print(class_names)


['curls' 'legs' 'no_pose']


In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [12]:
tf.autograph.experimental.set_loop_options(
    shape_invariants=[(outputs, tf.TensorShape([None, 2]))])


In [18]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    batch_size=16,
                    epochs=200,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])

--------------TRAINING----------------
Epoch 1/200
145/150 [============================>.] - ETA: 0s - loss: 0.0420 - accuracy: 0.9858
Epoch 1: val_accuracy did not improve from 0.99289
150/150 [==============================] - 1s 4ms/step - loss: 0.0460 - accuracy: 0.9853 - val_loss: 0.0546 - val_accuracy: 0.9905
Epoch 2/200
142/150 [===========================>..] - ETA: 0s - loss: 0.0424 - accuracy: 0.9872
Epoch 2: val_accuracy did not improve from 0.99289
150/150 [==============================] - 0s 3ms/step - loss: 0.0417 - accuracy: 0.9870 - val_loss: 0.0720 - val_accuracy: 0.9834
Epoch 3/200
146/150 [============================>.] - ETA: 0s - loss: 0.0482 - accuracy: 0.9842
Epoch 3: val_accuracy did not improve from 0.99289
150/150 [==============================] - 0s 2ms/step - loss: 0.0474 - accuracy: 0.9845 - val_loss: 0.0584 - val_accuracy: 0.9858
Epoch 4/200
147/150 [============================>.] - ETA: 0s - loss: 0.0489 - accuracy: 0.9813
Epoch 4: val_accuracy did n

129/150 [========================>.....] - ETA: 0s - loss: 0.0470 - accuracy: 0.9826
Epoch 30: val_accuracy did not improve from 0.99289
150/150 [==============================] - 0s 2ms/step - loss: 0.0445 - accuracy: 0.9837 - val_loss: 0.0684 - val_accuracy: 0.9834
Epoch 31/200
149/150 [============================>.] - ETA: 0s - loss: 0.0289 - accuracy: 0.9904
Epoch 31: val_accuracy did not improve from 0.99289
150/150 [==============================] - 0s 2ms/step - loss: 0.0288 - accuracy: 0.9904 - val_loss: 0.0674 - val_accuracy: 0.9905
Epoch 32/200
144/150 [===========================>..] - ETA: 0s - loss: 0.0467 - accuracy: 0.9848
Epoch 32: val_accuracy did not improve from 0.99289
150/150 [==============================] - 0s 2ms/step - loss: 0.0464 - accuracy: 0.9841 - val_loss: 0.0725 - val_accuracy: 0.9858
Epoch 33/200
139/150 [==========================>...] - ETA: 0s - loss: 0.0314 - accuracy: 0.9892
Epoch 33: val_accuracy did not improve from 0.99289
150/150 [===========

In [19]:
print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)

-----------------EVAUATION----------------
78/78 [==============================] - 0s 1ms/step - loss: 0.8134 - accuracy: 0.8601
LOSS:  0.8133879899978638
ACCURACY:  0.8600806593894958


# Save the CNN as pickle for deployment

In [17]:
model.save('pretrained_model24.h5')
